In [1]:
%matplotlib inline

In [2]:
import model_utils
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [3]:
data_dir = 'data/masked_images/data/trainval/species/'
test_data_dir = 'data/masked_images/data/test/species/'
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
test_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
trainloader, valloader, classes, labels = model_utils.load_split_train_val(data_dir, train_transforms, test_transforms, normalize,.25)

In [4]:
use_gpu = torch.cuda.is_available()

model_species = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_species.fc.in_features

model_species.fc = nn.Linear(num_ftrs, len(classes))

if use_gpu:
    model_species = model_species.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_species.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [6]:
model_species = model_utils.train_model(model=model_species,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=scheduler,
                           trainloader=trainloader,
                           valloader=valloader,
                           num_epochs=5)

Epoch 0/4
--------------------
train Loss: 0.4299 Acc: 0.7969
val Loss: 0.2394 Acc: 0.9166
--------------------
Epoch 1/4
--------------------
train Loss: 0.0685 Acc: 0.9826
val Loss: 0.0503 Acc: 0.9790
--------------------
Epoch 2/4
--------------------
train Loss: 0.0245 Acc: 0.9939
val Loss: 0.3072 Acc: 0.9098
--------------------
Epoch 3/4
--------------------
train Loss: 0.0141 Acc: 0.9959
val Loss: 0.6417 Acc: 0.8610
--------------------
Epoch 4/4
--------------------
train Loss: 0.0067 Acc: 0.9991
val Loss: 0.4179 Acc: 0.9024
--------------------
Best val Acc: 0.978983


In [7]:
saving_path = './trained_model/resnet50_species.pth'
torch.save(model_species, saving_path)

In [8]:
model_utils.print_test_acc(model_species, test_transforms, normalize, test_data_dir)

Overall accuracy for all breeds 90% 

Accuracy of   cat : 100 %
Accuracy of   dog : 85 %
